In [866]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [867]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [868]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[14,2]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    self.a.assign(output)
    return self.a.value()

In [869]:
class Layer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(Layer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val
    if self.name in ["HiddenLayer"]:
      self.type = "Hidden"
    elif self.name in ["OutputLayer"]:
      self.type = "Output"

  def build(self, input_shape):
    shape = [int(input_shape[-1]),self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
    
    if self.type == "Hidden":
      net = tf.math.reduce_sum(inputs * tf.transpose(self.w), axis=)
    elif self.type == "Output":
      net = tf.matmul(inputs, tf.transpose(self.w)) # possibile transposizione di w T ON
    # i pesi di un neurone sono considerati in riga
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


### Testing foward pass

In [870]:
x = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
#x = tf.constant([0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0])
x

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=float32)>

In [871]:
input_layer = InputLayer(name = "InputLayer")
input_layer

In [872]:
res_input = input_layer(x)
res_input

<tf.Tensor: shape=(14, 2), dtype=float32, numpy=
array([[0.3, 0.5],
       [0.4, 0.6],
       [0.1, 0.6],
       [0.2, 0.3],
       [0.6, 0.7],
       [0.2, 0.6],
       [0.4, 0.8],
       [0.4, 0.2],
       [0.1, 0.4],
       [0.1, 0.2],
       [0.3, 0.4],
       [0.1, 0.3],
       [0.3, 0.7],
       [0.1, 0.5]], dtype=float32)>

In [873]:
input_layer.a

<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32, numpy=
array([[0.3, 0.5],
       [0.4, 0.6],
       [0.1, 0.6],
       [0.2, 0.3],
       [0.6, 0.7],
       [0.2, 0.6],
       [0.4, 0.8],
       [0.4, 0.2],
       [0.1, 0.4],
       [0.1, 0.2],
       [0.3, 0.4],
       [0.1, 0.3],
       [0.3, 0.7],
       [0.1, 0.5]], dtype=float32)>

In [874]:
hidden_layer = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.0)
hidden_layer

In [875]:
res_hidden = hidden_layer(inputs = res_input, temperature = 1)
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [876]:
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5]], dtype=float32)>

In [877]:
hidden_layer.a

<tf.Variable 'HiddenLayer/HiddenLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [878]:
output_layer = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.0)
output_layer

In [879]:
res_output = output_layer(inputs = res_hidden, temperature = 1)
res_output

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [880]:
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
     

In [881]:
output_layer.a

<tf.Variable 'OutputLayer/OutputLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [882]:
error = tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
error

<tf.Tensor: shape=(), dtype=float32, numpy=2.2692394>

### Test back propagation

In [883]:
alpha = 1

In [884]:
delta_Ao = res_output * (res_hidden - res_output) * (1 - res_output) + output_layer.deltaA * (res_hidden - res_output)
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00293398, -0.00275687, -0.0030239 , -0.003206  , -0.00249945,
        -0.00293398, -0.00258405, -0.00311461, -0.003206  , -0.00339037,
        -0.0030239 , -0.00329795, -0.00275687, -0.00311461]],
      dtype=float32)>

In [885]:
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [886]:
tf.transpose(res_hidden)

<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.59868765],
       [0.62245935],
       [0.5866176 ],
       [0.5621765 ],
       [0.65701044],
       [0.59868765],
       [0.6456563 ],
       [0.5744425 ],
       [0.5621765 ],
       [0.5374298 ],
       [0.5866176 ],
       [0.54983395],
       [0.62245935],
       [0.5744425 ]], dtype=float32)>

In [887]:
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00293398, -0.00275687, -0.0030239 , -0.003206  , -0.00249945,
        -0.00293398, -0.00258405, -0.00311461, -0.003206  , -0.00339037,
        -0.0030239 , -0.00329795, -0.00275687, -0.00311461]],
      dtype=float32)>

In [888]:
delta_Wo = alpha * tf.matmul(tf.transpose(res_hidden),delta_Ao)   #dubbio scambiare righe e colonne
delta_Wo

<tf.Tensor: shape=(14, 14), dtype=float32, numpy=
array([[-0.00175654, -0.0016505 , -0.00181037, -0.00191939, -0.00149639,
        -0.00175654, -0.00154704, -0.00186468, -0.00191939, -0.00202977,
        -0.00181037, -0.00197444, -0.0016505 , -0.00186468],
       [-0.00182628, -0.00171604, -0.00188226, -0.0019956 , -0.00155581,
        -0.00182628, -0.00160846, -0.00193872, -0.0019956 , -0.00211037,
        -0.00188226, -0.00205284, -0.00171604, -0.00193872],
       [-0.00172112, -0.00161723, -0.00177387, -0.00188069, -0.00146622,
        -0.00172112, -0.00151585, -0.00182709, -0.00188069, -0.00198885,
        -0.00177387, -0.00193464, -0.00161723, -0.00182709],
       [-0.00164941, -0.00154985, -0.00169997, -0.00180234, -0.00140513,
        -0.00164941, -0.00145269, -0.00175096, -0.00180234, -0.00190599,
        -0.00169997, -0.00185403, -0.00154985, -0.00175096],
       [-0.00192765, -0.00181129, -0.00198674, -0.00210637, -0.00164217,
        -0.00192765, -0.00169775, -0.00204633, -0

In [889]:
output_layer.w.assign_add(delta_Wo)
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.5882434 , 0.58834946, 0.5881896 , 0.5880806 , 0.5885036 ,
        0.5882434 , 0.58845294, 0.5881353 , 0.5880806 , 0.5879702 ,
        0.5881896 , 0.5880255 , 0.58834946, 0.5881353 ],
       [0.5881737 , 0.58828396, 0.5881177 , 0.58800435, 0.5884442 ,
        0.5881737 , 0.5883915 , 0.5880613 , 0.58800435, 0.5878896 ,
        0.5881177 , 0.58794713, 0.58828396, 0.5880613 ],
       [0.58827883, 0.5883827 , 0.5882261 , 0.58811927, 0.58853376,
        0.58827883, 0.5884841 , 0.5881729 , 0.58811927, 0.58801115,
        0.5882261 , 0.5880653 , 0.5883827 , 0.5881729 ],
       [0.58835053, 0.58845013, 0.5883    , 0.58819765, 0.58859485,
        0.58835053, 0.5885473 , 0.588249  , 0.58819765, 0.588094  ,
        0.5883    , 0.588146  , 0.58845013, 0.588249  ],
       [0.5880723 , 0.5881887 , 0.58801323, 0.5878936 , 0.5883578 ,
        0.5880723 , 0.58830225, 0.5879536 , 0.5878936 , 0.5877725 ,
      

In [890]:
delta_Ah = res_hidden * (1 - res_hidden) * (tf.matmul(delta_Ao , output_layer.w)) # dubbio scambiare righe per colonne
delta_Ah

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00591398, -0.00578561, -0.0059685 , -0.00605691, -0.00554931,
        -0.00591398, -0.00563346, -0.00601622, -0.00605691, -0.00611644,
        -0.0059685 , -0.00609037, -0.00578561, -0.00601622]],
      dtype=float32)>

In [891]:
delta_Wh = tf.transpose(alpha * tf.transpose(delta_Ah) * res_input)
delta_Wh

<tf.Tensor: shape=(2, 14), dtype=float32, numpy=
array([[-0.0017742 , -0.00231424, -0.00059685, -0.00121138, -0.00332959,
        -0.0011828 , -0.00225339, -0.00240649, -0.00060569, -0.00061164,
        -0.00179055, -0.00060904, -0.00173568, -0.00060162],
       [-0.00295699, -0.00347137, -0.0035811 , -0.00181707, -0.00388452,
        -0.00354839, -0.00450677, -0.00120324, -0.00242276, -0.00122329,
        -0.0023874 , -0.00182711, -0.00404993, -0.00300811]],
      dtype=float32)>

In [892]:
hidden_layer.w.assign_add(delta_Wh)
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.4982258 , 0.49768576, 0.49940315, 0.49878863, 0.49667042,
        0.4988172 , 0.49774662, 0.49759352, 0.4993943 , 0.49938837,
        0.49820945, 0.49939096, 0.4982643 , 0.49939838],
       [0.497043  , 0.49652863, 0.4964189 , 0.49818292, 0.49611548,
        0.49645162, 0.49549323, 0.49879676, 0.49757725, 0.4987767 ,
        0.4976126 , 0.49817288, 0.49595007, 0.4969919 ]], dtype=float32)>

---

# CUSTOM MODEL

In [893]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.1)
    self.o = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.1)
    self.temperature = tf.Variable(initial_value=1.0,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))
    print("Input layer activations")
    print(self.input_layer.a)

    ### BACK PROPAGATION
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao) #possibile T della mat mult
    self.o.deltaA.assign(delta_Ao)
    self.o.w.assign_add(delta_Wo)

    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.reshape( ( tf.reduce_sum((tf.transpose(self.o.deltaA) * self.o.w), axis= 1)), shape = [1, 14] ) #T di w
    delta_Wh = tf.transpose(self.alpha * tf.transpose(delta_Ah) * self.input_layer.a)
    self.h.deltaA.assign(delta_Ah)
    self.h.w.assign_add(delta_Wh)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [894]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.224, 0.875, 0.266, 0.427, 0.168])
#input = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
model = MQSelfReflexiveNetwork(learning_rate=10)

In [895]:
model.compile()
model_history = model.fit(x = input, epochs=500)

Epoch 1/500
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
1/1 [==============================] - 0s 349ms/step - Error: 2.3268 - Temperature: 0.6994 - Hidden weights: 0.4876 - Output weights: 0.5735
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - Error: 2.0037 - Temperature: 0.6671 - Hidden weights: 0.4910 - Output weights: 0.5785
Epoch 3/500
1/1 [==============================] - 0s 6ms/step - Error: 2.2079 - Temperature: 0.6883 - Hidden weights: 0.4893 - Output weights: 0.5753
Epoch 4/500
1/1 [==============================] - 0s 6ms/step - Error: 1.8888 - Temperature: 0.6538 - Hidden weights: 0.4893 - Output weights: 0.5754
Epoch 5/500
1/1 [==============================] - 0s 2ms/step - Error: 1.9077 - Temperature: 0.6561 - Hidden weights: 0.4888 - Output weights: 0.5746
Epoch 6/500
1/1 [==============================] - 0s 2ms/s

In [896]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,2.326848,0.699415,"[[0.49935102, 0.49861756, 0.48906678, 0.476027...","[[0.5729911, 0.5769441, 0.5779243, 0.57491255,..."
1,2.003731,0.667081,"[[0.50637114, 0.49877313, 0.4894793, 0.4836204...","[[0.5788427, 0.58207905, 0.5824776, 0.5785961,..."
2,2.207851,0.688265,"[[0.5060303, 0.49518993, 0.48937613, 0.4820653...","[[0.5753949, 0.57933635, 0.57934797, 0.5758180..."
3,1.888808,0.653836,"[[0.50629693, 0.4951424, 0.48937947, 0.4820630...","[[0.5755971, 0.57923007, 0.57935363, 0.5757431..."
4,1.907749,0.656091,"[[0.50626737, 0.49465114, 0.4891177, 0.4819763...","[[0.57472056, 0.57877123, 0.57878935, 0.574900..."
...,...,...,...,...
495,0.000000,0.000000,"[[2.3200026, 1.2941734, 0.3502656, 0.7153834, ...","[[-0.7651014, -0.07697371, -7.9409738, -0.3904..."
496,0.000000,0.000000,"[[2.3200026, 1.2941734, 0.3502656, 0.7153834, ...","[[-0.7651014, -0.07697371, -7.9409738, -0.3904..."
497,0.000000,0.000000,"[[2.3200026, 1.2941734, 0.3502656, 0.7153834, ...","[[-0.7651014, -0.07697371, -7.9409738, -0.3904..."
498,0.000000,0.000000,"[[2.3200026, 1.2941734, 0.3502656, 0.7153834, ...","[[-0.7651014, -0.07697371, -7.9409738, -0.3904..."


In [897]:
model.get_weights()[1][0]

array([2.3200026 , 1.2941734 , 0.3502656 , 0.7153834 , 0.6021421 ,
       0.96267825, 2.149321  , 1.3145874 , 0.6558201 , 1.2273263 ,
       0.24240296, 0.48160884, 1.3265692 , 1.6761127 ], dtype=float32)

In [898]:
model.get_weights()[1][1]

array([ 0.9868816 ,  0.79915005, -1.2895613 ,  1.3333063 , -1.4739394 ,
        2.0847092 , -0.55022055, -1.2085537 , -2.8721857 ,  1.0567716 ,
       -0.5570395 , -1.4716983 ,  0.10168675,  0.07382874], dtype=float32)

In [899]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([1.333121  , 0.4950233 , 1.6398269 , 0.6179229 , 2.0760815 ,
       1.122031  , 2.6995416 , 2.523141  , 3.5280058 , 0.17055464,
       0.79944247, 1.9533072 , 1.2248825 , 1.602284  ], dtype=float32)

In [900]:
model.get_weights()[0]

array([[0.224, 0.266],
       [0.427, 0.   ],
       [0.01 , 0.   ],
       [0.875, 0.224],
       [0.   , 0.168],
       [0.875, 0.   ],
       [0.427, 0.03 ],
       [0.427, 0.875],
       [0.01 , 0.427],
       [0.01 , 0.875],
       [0.224, 0.427],
       [0.01 , 0.224],
       [0.224, 0.168],
       [0.01 , 0.266]], dtype=float32)

In [901]:
model.get_weights()[0][:,0]

array([0.224, 0.427, 0.01 , 0.875, 0.   , 0.875, 0.427, 0.427, 0.01 ,
       0.01 , 0.224, 0.01 , 0.224, 0.01 ], dtype=float32)

In [902]:
model.get_weights()[0][:,1]

array([0.266, 0.   , 0.   , 0.224, 0.168, 0.   , 0.03 , 0.875, 0.427,
       0.875, 0.427, 0.224, 0.168, 0.266], dtype=float32)

In [903]:
dva = abs(model.get_weights()[0][:,0] - model.get_weights()[0][:,1])
dva

array([0.042     , 0.427     , 0.01      , 0.651     , 0.168     ,
       0.875     , 0.39699998, 0.448     , 0.417     , 0.865     ,
       0.20299998, 0.214     , 0.05600001, 0.256     ], dtype=float32)

In [904]:
dvf

array([1.333121  , 0.4950233 , 1.6398269 , 0.6179229 , 2.0760815 ,
       1.122031  , 2.6995416 , 2.523141  , 3.5280058 , 0.17055464,
       0.79944247, 1.9533072 , 1.2248825 , 1.602284  ], dtype=float32)

In [905]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Monitored variables",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.show()

---